In [1]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [2]:
!pip install -U ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.2/611.2 kB 11.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json
from time import sleep
from PIL import Image, ImageDraw
import io
import base64
import requests
from os.path import exists
import os, sys, re, glob

from ultralytics import YOLO

import cv2
from google.colab.patches import cv2_imshow

import numpy as np

import shutil

import gradio as gr

In [7]:
%cd /content/
!mkdir model_pt # 모델2(위반사항 탐지 모델) 불러올 폴더 생성
!cp /content/drive/MyDrive/USG/yolo_best.pt /content/model_pt
model = YOLO('/content/model_pt/yolo_best.pt')

/content


In [63]:
# gradio!!!!
from IPython.display import Image

def detect(image):
    results = model(image)
    for i, result in enumerate(results):
      class_id = result.boxes.cls.int().tolist()
      N = len(class_id)
      if(N):
        idx_to_classes = results[0].names
        boxes = result.boxes.xyxy.int().tolist()
        conf  = result.boxes.conf.tolist()

        # 가로등 또는 오토바이 탐지 시 resnet으로 넘기면 안된다.
        tmp1, tmp2, tmp3 = [],[],[]
        for idx in range(N):
          ## xyxy[0] : 좌상 x, xyxy[1] : 좌상 y, xyxy[2] : 우하 x, xyxy[3] : 우하 y
          xyxy = boxes[idx]
          if np.abs(xyxy[2] - xyxy[0]) > 120 and np.abs(xyxy[3] - xyxy[1]) > 120:
            tmp1.append(class_id[idx])
            tmp2.append(boxes[idx])
            tmp3.append(conf[idx])
        class_id = tmp1
        boxes = tmp2
        conf = tmp3

        # 하나의 객체에 대해 중복되는 박스를 하나만 남긴다.
        # 중복 바운딩 박스 제거
        tmp1 = []
        rem = []
        visited = [False] * len(class_id)
        M = 0
        flag = False
        for p in np.arange(len(class_id)):
          for q in np.arange(p+1, len(class_id)):
            if visited[q] == False and  np.abs(boxes[p][0]-boxes[q][0]) < 50 and np.abs(boxes[p][1] - boxes[q][1]) < 50:
              visited[q] = True
              tmp1.append([class_id[q], boxes[q], conf[q]])
              M += 1
              if M == len(class_id)-1:
                flag = True
                break
          if flag:
            break

        for item in tmp1:
          class_id.remove(item[0])
          boxes.remove(item[1])
          conf.remove(item[2])

        img = image
        for j, xyxy in enumerate(boxes):
            x1, y1, x2, y2 = xyxy
            img = cv2.rectangle(img, (x1,y1),(x2,y2), (255,255,255), 3)
            img = cv2.putText(img,  idx_to_classes[class_id[j]], (x1, y1-20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
            img = cv2.putText(img, f'conf: {conf[j]:.4f}', (x1, y1-65), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
        #cv2_imshow(img)

        return img


demo = gr.Interface(detect,
                    inputs= [gr.Image()],
                    outputs=[gr.Image()]
                    )

if __name__ == "__main__":
    #demo.queue()
    demo.launch(debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>